# postgreSQL 연결

In [1]:
import psycopg2

In [2]:
conn_string = "host='localhost' dbname = 'seoul_dust' user= 'postgres' password='10co2pt' port = '5432'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

In [3]:
# cur.execute("CREATE TABLE dust_20090102 (MSRDT real, MSRSTE_NIM text, NO2 real, O3 real, CO real, SO2 real, PM10 real, PM25 real);")

In [11]:
# conn.commit()

In [11]:
# cur.execute("COPY dust_20090102 FROM 'D:/DataScience/first-repository/raw_data/dust_20090102.csv' DELIMITER ',' CSV HEADER;")

In [12]:
# conn.commit()

# open API 데이터 수집

In [3]:
import requests
import pandas as pd
import os
import datetime as dt
import matplotlib.pyplot as plt
import urllib3

from time import sleep

In [4]:
def call_api(api_name, start_date, end_date, dir_name):

    # API 키는 공개하기 힘든 점 양해 바랍니다.

    api_key = open("./api_key/SEOUL_api_key.txt").readlines()[0].strip()

    url_format = 'http://openAPI.seoul.go.kr:8088/{api_key}/json/{api_name}/1/{end_index}/{date}'

    headers = {'content-type': 'application/json;charset=utf-8'}



    for date in pd.date_range(start_date, end_date).strftime("%Y%m%d"):

        # 최초 1회 Call은 해당 일자의 데이터 수를 확인한다.

        url = url_format.format(api_name=api_name, api_key=api_key, end_index=1, date=date)

        response = requests.get(url, headers=headers)

        end_index = response.json()[api_name]["list_total_count"]

        print("Max Count(%s): %s" % (date, end_index))



        # 해당 일자의 모든 데이터를 불러온다.

        url = url_format.format(api_name=api_name, api_key=api_key, end_index=end_index, date=date)

        response = requests.get(url, headers=headers)

        result = pd.DataFrame(response.json()[api_name]["row"])

        #cur.execute("CREATE TABLE dust_%s (MSRDT real, MSRSTE_NIM text, NO2 real, O3 real, CO real, SO2 real, PM10 real, PM25 real);" % (date))
        #conn.commit()
        
        # PSQL에 적재 시, UTF-8 / R에서 read.table에서는 EUC-KR
        result.to_csv("./raw_data/%s/dust_%s.csv" % (dir_name, date), index=False, encoding="EUC-KR") #"UTF-8") 
        #result.to_csv("./raw_data/%s/dust.csv" % (dir_name), index=False, encoding="UTF-8") #"EUC-KR") #"UTF-8") 
        
        #cur.execute("COPY dust_%s FROM 'D:/DataScience/first-repository/raw_data/%s/dust_%s.csv' with DELIMITER ',' CSV HEADER;" % (date, dir_name, date))
        #cur.execute("COPY dust_%s FROM './raw_data/%s/dust.csv' with DELIMITER ',' CSV HEADER;" % (date, dir_name))
        



        # API 부하 관리를 위해 0.5초 정도 쉬어 줍시다 (찡긋)

        sleep(0.5)

In [3]:
call_api("TimeAverageAirQuality", "2019-01-01", "2020-08-31", "dust_init")

Max Count(20190101): 600
Max Count(20190102): 600
Max Count(20190103): 600
Max Count(20190104): 600
Max Count(20190105): 600
Max Count(20190106): 600
Max Count(20190107): 600
Max Count(20190108): 600
Max Count(20190109): 600
Max Count(20190110): 600
Max Count(20190111): 600
Max Count(20190112): 600
Max Count(20190113): 600
Max Count(20190114): 600
Max Count(20190115): 600
Max Count(20190116): 600
Max Count(20190117): 600
Max Count(20190118): 600
Max Count(20190119): 600
Max Count(20190120): 600
Max Count(20190121): 600
Max Count(20190122): 600
Max Count(20190123): 600
Max Count(20190124): 600
Max Count(20190125): 600
Max Count(20190126): 600
Max Count(20190127): 600
Max Count(20190128): 600
Max Count(20190129): 600
Max Count(20190130): 600
Max Count(20190131): 600
Max Count(20190201): 600
Max Count(20190202): 600
Max Count(20190203): 600
Max Count(20190204): 600
Max Count(20190205): 600
Max Count(20190206): 600
Max Count(20190207): 600
Max Count(20190208): 600
Max Count(20190209): 600


In [45]:
for date_temp in pd.date_range('20100101', '20200102').strftime("%Y%m%d"):
    conn.commit()
    cur.execute("COPY (SELECT * FROM dust_%s) TO 'D:/DataScience/first-repository/raw_data/dust/dust_%s.csv' WITH CSV DELIMITER ',' encoding 'EUC-KR';" % (date_temp, date_temp))

In [30]:
conn.commit()
cur.execute("COPY (SELECT * FROM dust_20100101) TO 'D:/DataScience/first-repository/raw_data/dust/dust_20100101.csv' WITH CSV DELIMITER ',' encoding 'EUC-KR';")